In [2]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input

In [ ]:
model = keras.models.load_model('best_model_06_0.985.h5')

In [4]:
# prepare sample image
path = 'sample_without_mask.jpg'
img = load_img(path, target_size=(100,100))
x = np.array(img)
X = np.array([x])
X = preprocess_input(X)

In [6]:
pred = model.predict(X)

In [11]:
pred

# Correct: Remember that without mask: 1

array([[0.99996793]], dtype=float32)

## Keras to TFlite

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('best_model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

In [14]:
import tensorflow.lite as tflite


In [15]:
interpreter = tflite.Interpreter(model_path='best_model.tflite')
interpreter.allocate_tensors()
interpreter.get_input_details()

[{'name': 'serving_default_conv2d_input:0',
  'index': 0,
  'shape': array([  1, 100, 100,   3], dtype=int32),
  'shape_signature': array([ -1, 100, 100,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [16]:
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [18]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
pred = interpreter.get_tensor(output_index)
pred

!! Correct and same with above: Remember that without mask: 1

### Removing TensorFlow dependency

In [47]:
from PIL import Image

with Image.open('sample_without_mask.jpg') as img:
    img = img.resize((100, 100), Image.NEAREST)

x = np.array(img, dtype='float32')
X = np.array([x])

X /= 255

In [50]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
pred = interpreter.get_tensor(output_index)
pred

array([[0.9977058]], dtype=float32)

!! Correct and same with above: Remember that without mask: 1